In [1]:
# packages 
from dotenv import load_dotenv
import os
from openai import OpenAI, AuthenticationError

# widgets 
from widgets import Chat, ChatInput, Message

In [2]:
load_dotenv()

api_key = os.environ["OPENAI_KEY"]

def create_client(api_key):
    try:
        client = OpenAI(api_key=api_key)
        client.models.list()
        return client
    except AuthenticationError:
        print("Incorrect API")
    return None

client = create_client(api_key)

In [3]:
history = []

In [4]:
chat = Chat()

Output()

In [5]:
chat_input = ChatInput()

ChatInputWidget()

In [9]:
msg = chat_input.submitted.strip()

if msg:
    # user
    user_msg = Message(role="user", emoji="👤")
    user_msg.set_message(markdown=msg)
    chat.add(user_msg)

    # ai
    bot_msg = Message(role="assistant", emoji="🤖")
    chat.add(bot_msg)

    # messages history
    history.append({"role": "user", "content": msg})

    # streaming responses
    with client.responses.stream(
        model="gpt-4.1",
        input=history,
    ) as stream:
        full_reply = ""
        for event in stream:
            if event.type == "response.output_text.delta":
                bot_msg.append_markdown(event.delta)
                full_reply += event.delta

        # add response to history 
        final = stream.get_final_response()
        history.append({"role": "assistant", "content": full_reply})